In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import pca_pseudo_grid_search

#otros
import os
import numpy as np
import pandas as pd
import timeit
from sklearn.externals import joblib

tipo = pruebas[3]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Pruebas con el dataset Amazon

In [3]:
print tipo
print dataset_name
print dims
print data_path

pca
amazon
3000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print X.shape

print "Todos los datos disponibles obtenidos"

(27677, 3000)
Todos los datos disponibles obtenidos


# Adaptación


## Creación de modelos de adaptación.

Para cada dominio se entrenan distintos modelos según los parámetros enviados.

Cada modelo es guardado en la ruta: models/amazon/pca/me1\_[dominio\_objetivo]_[numero_de_modelo].pkl

Todas las rutas son guardadas en una lista, la cual es almacenada en: models/amazon/pca/me1_models_paths.pkl

```python
paths_list = [ruta1, ..., ruta_n]
```

In [6]:
models_paths = os.path.join(models_path, dataset_name, tipo, "me1_models_paths.pkl")
paths_list = []

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):
    print "Cargando rutas de modelos adaptados."
    paths_list = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_list'"
# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parametros = {
        'n_components': [int(dims/2), int(dims/3), int(dims/4)]
    }

    print "Creando modelos de adaptacion..."
    
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me1_"
    
    #paths_list = msda_theano_pseudo_grid_search(X, parametros, folder_path, prefix)
    paths_list = pca_pseudo_grid_search(X, parametros, folder_path, prefix)
    
    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_list, models_paths)
    print "Rutas cargadas en la variable 'paths_list'"

Creando modelos de adaptacion...
	n_components: 1500
	Entrenando modelo PCA...
	Guardando modelo en models/amazon/pca/me1_0.pkl

	n_components: 1000
	Entrenando modelo PCA...
	Guardando modelo en models/amazon/pca/me1_1.pkl

	n_components: 750
	Entrenando modelo PCA...
	Guardando modelo en models/amazon/pca/me1_2.pkl


Creacion de modelos terminada
Guardando rutas en models/amazon/pca/me1_models_paths.pkl
Rutas cargadas en la variable 'paths_list'


In [7]:
print "Modelos almacenados en:"
for ruta in paths_list:
    print "\t", ruta

Modelos almacenados en:
	models/amazon/pca/me1_0.pkl
	models/amazon/pca/me1_1.pkl
	models/amazon/pca/me1_2.pkl


```python
domain_msda_paths = [ruta_1,...,ruta_n]
```

## Busqueda del mejor modelo por dominio

Por cada dominio se busca el mejor modelo de adaptación.

Esto se obtiene adaptando los datos de entrenamiento de cada dominio por cada modelo creado y realizando Grid-Search y Cross-Validation con estos datos.

El modelo que logre un mejor valor de Cross-Validation es considerado el mejor modelo para adaptar y queda guardado en un diccionario de la forma:

```python
best_models = {
    [dominio_1] = ruta,
    .
    .
    .
    [dominio_n] = ruta,
}
```

Este diccionario queda almacenado en la ruta:
    models/amazon/pca/me1_best_models.pkl

In [8]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_best_models.pkl")
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"  
else:
    print "Obteniendo mejores modelos..."
    for domain in domains:
        print domain
        i = 1
        best_score = 0
        best_path = None
        best_model = None
        
        for pca_model_path in paths_list:
            
            pca_model = joblib.load(pca_model_path)
            n_components = pca_model.n_components_
            
            print "%d) n_components=%d " % (i, n_components),
            # se obtienen los datos del dominio
            X_tr = np.asarray(labeled[domain]['X_tr'].todense())
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = pca_model.transform(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_path = pca_model_path
                best_model = pca_model
            i = i+1
        
        #se guarda el mejor modelo para este dominio
        print "Mejor modelo: n_components=%d" % (best_model.n_components_)
        best_models[domain] = best_path
        
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
electronics
1) n_components=1500  score: 0.8887
2) n_components=1000  score: 0.8895
3) n_components=750  score: 0.8831
Mejor modelo: n_components=1000
dvd
1) n_components=1500  score: 0.8263
2) n_components=1000  score: 0.8263
3) n_components=750  score: 0.8101
Mejor modelo: n_components=1000
kitchen
1) n_components=1500  score: 0.9092
2) n_components=1000  score: 0.8973
3) n_components=750  score: 0.8919
Mejor modelo: n_components=1500
books
1) n_components=1500  score: 0.8241
2) n_components=1000  score: 0.8130
3) n_components=750  score: 0.7966
Mejor modelo: n_components=1500
Rutas guardadas en  models/amazon/pca/me1_best_models.pkl


In [9]:
for dominio, ruta in best_models.items():
    print "%s - %s" % (dominio, ruta)

dvd - models/amazon/pca/me1_1.pkl
electronics - models/amazon/pca/me1_1.pkl
books - models/amazon/pca/me1_0.pkl
kitchen - models/amazon/pca/me1_0.pkl


In [10]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_model_path = best_models[domain]
    
    if os.path.exists(best_model_path):
        print "Cargando mejor modelo para %s" % domain
        pca_model = joblib.load(best_model_path)
        
        print "\tAdaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())

        tr_reps = pca_model.transform(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para electronics
	Adaptando dominio
Cargando mejor modelo para dvd
	Adaptando dominio
Cargando mejor modelo para kitchen
	Adaptando dominio
Cargando mejor modelo para books
	Adaptando dominio
4/4 dominios adaptados


## Pruebas de clasificación

In [11]:
df = pd.DataFrame(columns=dataframe_columns)

i=1
tareas = len(domains)*(len(domains)-1)

# por cada par posible para adaptar
for src in domains:
    #se carga el mejor modelo para el dominio fuente
    ruta = best_models[src]
    modelo = joblib.load(ruta)
    
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de %d" % (i, tareas)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### PCA ####
            #############
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = modelo.transform(X_ts)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i+=1

print "\nPruebas completadas"

Tarea 1 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 2 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 3 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 4 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 5 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 6 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 7 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 8 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 9 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 10 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 11 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 12 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.

Pruebas completadas


In [12]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
1,PCA,e->d,electronics,dvd,15.762894,28.985725,13.222831
2,PCA,e->k,electronics,kitchen,8.587715,11.537788,2.950074
3,PCA,e->b,electronics,books,15.157879,30.300758,15.142879
4,PCA,d->e,dvd,electronics,11.127778,29.908248,18.780470
5,PCA,d->k,dvd,kitchen,8.587715,20.803020,12.215305
6,PCA,d->b,dvd,books,15.157879,29.585740,14.427861
7,PCA,k->e,kitchen,electronics,11.127778,13.102828,1.975049
8,PCA,k->d,kitchen,dvd,15.762894,28.468212,12.705318
9,PCA,k->b,kitchen,books,15.157879,30.568264,15.410385
10,PCA,b->e,books,electronics,11.127778,21.590540,10.462762


In [13]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/amazon/pca/me1_3000.csv
Resultados guardados.
